# Requirements

In [1]:
import tenseal as ts # pip install tenseal
from deepface import DeepFace #!pip install deepface
import base64
from deepface.commons import distance as dst


# Finding embeddings (Client Side)
We are going to find vector representations of facial images. This will be done in the client side.

In [2]:
img1_path = "photo.jpeg"
img2_path = "new photo.jpeg"

In [3]:
img1_embedding=DeepFace.represent(img1_path, model_name='Facenet')
img2_embedding=DeepFace.represent(img2_path, model_name='Facenet')

In [4]:
len(img1_embedding)

1

# Commons

In [5]:
def write_data(file_name, file_content):
    
    if type(file_content) == bytes:
        #bytes to base64
        
        file_content = base64.b64encode(file_content)
    
    with open(file_name, 'wb') as f: 
        f.write(file_content)

def read_data(file_name):
    with open(file_name, "rb") as f:
        file_content = f.read()
    
    #base64 to bytes
    return base64.b64decode(file_content)

# Initialize (Client Side)

In [6]:
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes = [60, 40, 40, 60])

In [7]:
context.generate_galois_keys()
context.global_scale = 2**40

In [8]:
secret_context = context.serialize(save_secret_key = True)
write_data("secret.txt", file_content=secret_context)

In [9]:
type(secret_context)

bytes

In [10]:
context.make_context_public()
public_context=context.serialize()
write_data(file_name="public.txt",file_content=public_context)

In [11]:
del context, secret_context, public_context

# Encryption
We are going to apply homomorphic encryption to facial embeddings. This will be done in the client side.

Then, homomorphic encrypted facial embeddings will be stored in the cloud.

In [ ]:
context = ts.context_from(read_data("secret.txt"))
enc_v1 = ts.ckks_vector(context, img1_embedding)
enc_v2 = ts.ckks_vector(context, img2_embedding)

In [ ]:
enc_v1_proto = enc_v1.serialize()
enc_v2_proto = enc_v2.serialize()

In [ ]:
write_data("enc_v1.txt", enc_v1_proto)
write_data("enc_v2.txt", enc_v2_proto)

In [ ]:
del context, enc_v1, enc_v2, enc_v1_proto, enc_v2_proto

# Calculations
Once homomorphic encrypted facial embeddings stored in the cloud, we are able to make calculations on encrypted data.

Notice that we just have public key here and we don't have secret key.

In [ ]:
context = ts.context_from(read_data("public.txt"))

In [ ]:
enc_v1_proto = read_data("enc_v1.txt")
enc_v2_proto = read_data("enc_v2.txt")

In [ ]:
enc_v1 = ts.lazy_ckks_vector_from(enc_v1_proto)
enc_v1.link_context(context)

enc_v2 = ts.lazy_ckks_vector_from(enc_v2_proto)
enc_v2.link_context(context)

In [ ]:
euclidean_squared = enc_v1 - enc_v2
euclidean_squared = euclidean_squared.dot(euclidean_squared)

In [ ]:
write_data("euclidean_squared.txt", euclidean_squared.serialize())

In [ ]:
#we must not decrypt the homomorphic encrypted euclidean squared value in this stage
#because we don't have the secret key. check this operation. it should throw an exception!

try:
    euclidean_squared.decrypt()
except Exception as err:
    print("Exception: ", str(err))

In [ ]:
del context, enc_v1_proto, enc_v2_proto, enc_v1, enc_v2, euclidean_squared

# Decryption
Once homomorphic encrypted euclidean squared value found in the cloud, we are going to retrieve it to the client side.

Client can decrypt it because we have the secret key in the client side.



In [ ]:
context = ts.context_from(read_data("secret.txt"))

In [ ]:
euclidean_squared_proto = read_data("euclidean_squared.txt")

In [ ]:
euclidean_squared = ts.lazy_ckks_vector_from(euclidean_squared_proto)
euclidean_squared.link_context(context)

In [ ]:
euclidean_squared_plain = euclidean_squared.decrypt()[0]

In [ ]:
euclidean_squared_plain

In [ ]:
if euclidean_squared_plain < 100:
    print("they are same person")
else:
    print("they are different persons")

# Validation
What if euclidean distance calculation is done in the client side always? Result should be same!

In [ ]:
distance = dst.findEuclideanDistance(img1_embedding, img2_embedding)


In [ ]:
print("euclidean squared - tradational: ", distance*distance)
print("euclidean squared - homomorphic: ", euclidean_squared_plain)

In [ ]:
#check the difference is acceptable
abs(distance * distance - euclidean_squared_plain) < 0.00001